In [0]:
from google.colab import files
import pandas as pd
import numpy as np
import json
import time
from textblob import TextBlob
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import nltk.stem as stem
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

In [0]:
start = time.time()
with open('/content/drive/My Drive/business.json') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data)) 
business=pd.DataFrame(data)
print("Time to load data: {} seconds".format(time.time() - start))
business.head()

Time to load data: 13.453125238418579 seconds


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [0]:
business.shape

(192609, 14)

In [0]:
start = time.time()
with open('/content/drive/My Drive/review.json') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data)) 
reviews=pd.DataFrame(data)
print("Time to load data: {} seconds".format(time.time() - start))


Time to load data: 216.22022151947021 seconds


In [0]:
reviews.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03


In [0]:
def getRestaurantsUSA(business):
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    df=business.loc[business['state'].isin(states)]
    return df
business=getRestaurantsUSA(business)
business.dropna(inplace=True)
business=business[business['categories'].str.contains('Restaurant[a-z]', case=False, regex=True)]
business.reset_index(drop=True)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
1,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W..."
2,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
3,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156,36.195615,-115.040529,4.5,184,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Mexican, Restaurants, Patisserie/Cake Shop, Fo...","{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'..."
4,BvYU3jvGd0TJ7IyZdfiN2Q,Manzetti's Tavern,6401 Morrison Blvd,Charlotte,NC,28211,35.156338,-80.831878,3.5,16,0,"{'Alcohol': 'u'full_bar'', 'BusinessAcceptsCre...","Sandwiches, Italian, American (Traditional), A...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27177,ngs16C2M_uTq2zXamltHVw,Jessie Rae's BBQ,5611 S Valley View Blvd,Las Vegas,NV,89118,36.087895,-115.190329,4.5,595,1,"{'Caters': 'True', 'RestaurantsTableService': ...","Farmers Market, Caterers, Food, Street Vendors...","{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:30'..."
27178,0fY-zYyP2fDmp2YXFsuNTg,Gotham Provisions Company,503 W Main St,Sun Prairie,WI,53590,43.182853,-89.220335,4.0,18,0,"{'OutdoorSeating': 'False', 'RestaurantsAttire...","Sandwiches, Food, Restaurants, Soup, Bagels","{'Tuesday': '7:0-14:0', 'Wednesday': '7:0-14:0..."
27179,JsRt9LPgv-7guVcY4u6OQA,Antico Italian Restaurant,"9719 Sam Furr Rd, Unit C",Huntersville,NC,28078,35.443723,-80.864550,4.5,142,1,"{'Ambience': '{'touristy': False, 'hipster': F...","Pizza, Italian, Restaurants, Seafood","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
27180,ghovD5ZTGDQ5Q2U4ERddWw,Five Guys,3863 Medina Rd,Fairlawn,OH,44333,41.136561,-81.639712,4.0,22,1,"{'RestaurantsPriceRange2': '2', 'HasTV': 'Fals...","Burgers, Restaurants, Fast Food, American (New)","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [0]:
business['categories'].to_list()

In [0]:
reviews=reviews.drop(columns=['stars'])

In [0]:
merged= business.merge(reviews, how='inner', on='business_id' )

In [0]:
merged.shape

(3255923, 21)

In [0]:
merged.groupby(['business_id']).size().reset_index(name='counts')

,business_id,counts
0,--9e1ONYQuAa-CB_Rrw7Tw,1661
1,--FBCX-N37CMYDfs790Bnw,142
2,--GM_ORV2cYS-h38DSaCLw,10
3,--I7YYLada0tSLkORTHb5Q,100
4,--KCl2FvVQpvjzmZSPyviA,15
...,...,...
27177,zzmIMvqiBJ_-wVKg_OnGpw,121
27178,zzsOLFhgUw8gnjLTVVItFA,107
27179,zzwhN7x37nyjP0ZM8oiHmw,57
27180,zzwicjPC9g246MK2M1ZFBA,76


In [0]:
merged['category']=pd.Series()
merged.loc[merged.categories.str.contains('American'),'category'] = 'American'
merged.loc[merged.categories.str.contains('Japanese'),'category'] = 'Japanese'
merged.loc[merged.categories.str.contains('Mexican'),'category'] = 'Mexican'
merged.loc[merged.categories.str.contains('Italian'),'category'] = 'Italian'
merged.loc[merged.categories.str.contains('Iranian'),'category'] = 'Iranian'
merged.loc[merged.categories.str.contains('Polish'),'category'] = 'Polish'
merged.loc[merged.categories.str.contains('Hawaiian'),'category'] = 'Hawaiian'
merged.loc[merged.categories.str.contains('Taiwanese'),'category'] = 'Taiwanese'
merged.loc[merged.categories.str.contains('Vietnamese'),'category'] = 'Vietnamese'
merged.loc[merged.categories.str.contains('Mediterranean'),'category'] = 'Mediterranean'
merged.loc[merged.categories.str.contains('Thai'),'category'] = 'Thai'
merged.loc[merged.categories.str.contains('French'),'category'] = 'French'
merged.loc[merged.categories.str.contains('Greek'),'category'] = 'Greek'
merged.loc[merged.categories.str.contains('Indian'),'category'] = 'Indian'
merged.loc[merged.categories.str.contains('Korean'),'category'] = 'Korean'
merged.loc[merged.categories.str.contains('African'),'category'] = 'African'
merged.loc[merged.categories.str.contains('Spanish'),'category'] = 'Spanish'

In [0]:
merged.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,review_id,user_id,useful,funny,cool,text,date,category
0,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",F7POrJsNbhu493DSTMPXjw,nsS4oDfOsl20QdWc6XcOkw,1,0,0,"Husband was craving Chicken Teriyaki & gyoza, ...",2014-02-24 02:51:56,Japanese
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",c-6aA9Bd7JxpmMroRoas9A,bK4Y_GZUoAUTXIrmeEUGYw,0,0,1,Went there Saturday noon they open at 12pm but...,2014-07-13 20:28:18,Japanese
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",P_MV_lMQjLIO9R7M_KIuMg,sbbB6Y4yIZIInvt0LFYYew,1,0,0,Went to this restaurant on Saturday as I had a...,2018-05-09 00:39:36,Japanese
3,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",dCLBdXP9tjjQG7EkHNeWzQ,hhQUVO2AqBdSeQcPo19Bsw,4,2,3,This place will always have a place in my stom...,2017-10-25 13:58:04,Japanese
4,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",GiBx0zRVxC1Bu-9R-MMF8w,XSz4svDvgOCRO92V_3ajWg,0,0,0,"Well first time I've really had ""authentic"" Ja...",2015-04-02 01:13:36,Japanese


In [0]:
merged['category'].unique()

array(['Japanese', 'Mexican', 'Italian', 'Iranian', 'Polish', 'American',
       nan, 'Thai', 'Korean', 'French', 'Mediterranean', 'Hawaiian',
       'Taiwanese', 'Vietnamese', 'Indian', 'Greek', 'African', 'Spanish'],
      dtype=object)

In [0]:
merged.dropna(inplace=True)

In [0]:
merged.groupby(['category']).size().reset_index(name='counts')

,category,counts
0,African,8319
1,American,964354
2,French,63608
3,Greek,59530
4,Hawaiian,40583
5,Indian,46834
6,Iranian,780
7,Italian,296406
8,Japanese,180693
9,Korean,61233


In [0]:
merged.drop(columns=['categories'], inplace=True)

In [0]:
merged.to_pickle('/content/drive/My Drive/merged.pkl')